In [10]:
import numpy as np
import time
import os
!pip install imutils
import subprocess
!pip install pydub

  Stored in directory: C:\Users\Sorya\AppData\Local\pip\Cache\wheels\16\84\1f\bf88641293cda2c8be81a5c4b8ca973dd9125a6dc3767417fd
Successfully built imutils


In [3]:
!pip install opencv-python  

In [13]:
import cv2


In [25]:
# load the COCO class labels our YOLO model was trained on
LABELS = open("data/coco.names").read().strip().split("\n")

# load our YOLO object detector trained on COCO dataset (80 classes)
net = cv2.dnn.readNetFromDarknet("cfg/yolov3.cfg", "yolov3.weights")

# determine only the *output* layer names that we need from YOLO
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# initialize
# load the image with imread()
imageSource = 'data/horses.jpg'
img = cv2.imread(imageSource)


# grab the image dimensions and convert it to a blob
(H, W) = img.shape[:2]
# construct a blob from the input image and then perform a forward
# pass of the YOLO object detector, giving us our bounding boxes and
# associated probabilities
blob = cv2.dnn.blobFromImage(img, 1 / 255.0, (416, 416),
swapRB=True, crop=False)
net.setInput(blob)
layerOutputs = net.forward(ln)

# initialize our lists of detected bounding boxes, confidences, and
# class IDs, respectively
boxes = []
confidences = []
classIDs = []
centers = []

# loop over each of the layer outputs
for output in layerOutputs:
  # loop over each of the detections
  for detection in output:
      # extract the class ID and confidence (i.e., probability) of
      # the current object detection
      scores = detection[5:]
      classID = np.argmax(scores)
      confidence = scores[classID]

      # filter out weak predictions by ensuring the detected
      # probability is greater than the minimum probability
      if confidence > 0.5:
          # scale the bounding box coordinates back relative to the
          # size of the image, keeping in mind that YOLO actually
          # returns the center (x, y)-coordinates of the bounding
          # box followed by the boxes' width and height
          box = detection[0:4] * np.array([W, H, W, H])
          (centerX, centerY, width, height) = box.astype("int")

          # use the center (x, y)-coordinates to derive the top and
          # and left corner of the bounding box
          x = int(centerX - (width / 2))
          y = int(centerY - (height / 2))

          # update our list of bounding box coordinates, confidences,
          # and class IDs
          boxes.append([x, y, int(width), int(height)])
          confidences.append(float(confidence))
          classIDs.append(classID)
          centers.append((centerX, centerY))
          if classID == 2:
            mid_x = (int(width))/2
            mid_y = (int(width))/2
            apx_distance = round(((1 - int(width))**4),1)
            print(apx_distance)


  # apply non-maxima suppression to suppress weak, overlapping bounding
  # boxes
  idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.3)

  texts = []

  # ensure at least one detection exists
  if len(idxs) > 0: 
    # loop over the indexes we are keeping
    for i in idxs.flatten():
      # find  
      centerX, centerY = centers[i][0], centers[i][1]

      if centerX <= W/3:
        W_pos = "left "
      elif centerX <= (W/3 * 2):
        W_pos = "center "
      else:
        W_pos = "right "

      texts.append(W_pos + LABELS[classIDs[i]])

print(texts)



['left horse', 'right horse', 'center horse']


In [22]:
!./darknet detect cfg/yolov3.cfg yolov3.weights data/dog.jpg


'.' is not recognized as an internal or external command,
operable program or batch file.


In [23]:
!darknet.exe detect cfg/yolov3.cfg yolov3.weights data/dog.jpg


'darknet.exe' is not recognized as an internal or external command,
operable program or batch file.
